<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/cloth_Baraff_Witkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install meshzoo

In [ ]:
import numpy as np
import meshzoo
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rc('animation', html='jshtml')

In [ ]:
vertex_positions_uv, all_triangle_indices = meshzoo.rectangle_tri(
    (0.0, 0.0),
    (1.0, 1.0),
    n=3,
    variant="zigzag",  # or "up", "down", "center"
)

print(vertex_positions_uv.shape)
print(all_triangle_indices.shape)

In [ ]:
x, y = np.transpose(vertex_positions_uv)
z = np.zeros_like(x)

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')

ax.plot_trisurf(x, y, z, triangles=all_triangle_indices)
ax.scatter(x, y, z, c='red', s=10, depthshade=False)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

In [ ]:
def triangle_area(triangle_vertices):
    v0, v1, v2 = triangle_vertices
    return np.linalg.norm(np.cross(v1 - v0, v2 - v0)) / 2.0

def calculate_area(triangles_indices, vertex_positions):
    area = 0.0
    for triangle_indices in all_triangle_indices:
        triangle_vertices = vertex_positions[triangle_indices]
        area += triangle_area(triangle_vertices)
    return area

In [ ]:
vertex_positions = np.transpose([x, y, z])
vertex_velocities = np.zeros_like(vertex_positions)
vertex_forces = np.zeros_like(vertex_velocities)

n = amount_of_particles = vertex_positions.shape[0]

# Later the mass should be calculated by looping over triangles and dividing
# the mass of the triangle between its vertices.
density = 0.2  # kg / m2
area = calculate_area(all_triangle_indices, vertex_positions)
mass = density * area
mass_per_particle = mass / n
masses = np.ones(n) * mass_per_particle

In [ ]:
# def evaluate_derivatives(positions, velocities):
#     return (velocities, forces / masses[:, np.newaxis])

In [ ]:
# evaluate_derivatives(positions, velocities);

# Scipy Sparse

In [ ]:
a = np.array([[1, 2], [3, 5]])
b = np.array([1, 2])
x = np.linalg.solve(a, b)
x

In [ ]:
a @ x

In [ ]:
import scipy.sparse.linalg

scipy.sparse.linalg.cg(a, b)

In [ ]:
A = np.zeros((3*n, 3*n))
x = np.zeros(3*n)
b = np.zeros(3*n)

In [ ]:
I = np.identity(3*n)
fps = 30.0
h = 1.0 / fps # 0.033 s = 33 ms
M_inverse = I / mass_per_particle

In [ ]:
for triangle_indices in all_triangle_indices:
    # given current vertex positions
    # find current stretch in u and v directions (w_u and w_v)
    # using the known rest state uv positions
    
    # i, j, k = triangle_indices

    # to avoid confusio between the x coordinate and the X position vector,
    # I'm going to capitalize the X when I mean the vector 
    # (position.x, position.y, position.z)
    # in the paper this is x printed in bold
    Xi, Xj, Xk = vertex_positions[triangle_indices]

    area = triangle_area(vertex_positions[triangle_indices])

    delta_X1 = Xj - Xi
    delta_X2 = Xk - Xi

    uv_i, uv_j, uv_k = vertex_positions_uv[triangle_indices]
    ui, vi = uv_i
    uj, vj = uv_j
    uk, vk = uv_k

    delta_u1 = uj - ui
    delta_u2 = uk - ui
    delta_v1 = vj - vi
    delta_v2 = vk - vi

    delta_u_matrix = np.array([(delta_u1, delta_u2),
                               (delta_v1, delta_v2)])
        
    delta_X_matrix = np.column_stack((delta_X1, delta_X2))

    # Equation (9) in Baraff-Witkin.
    # w_u = "world u"
    w_uv = delta_X_matrix @ np.linalg.inv(delta_u_matrix)

    wu, wv = np.hsplit(w_uv, 2)

    norm_wu = np.linalg.norm(wu)
    norm_wv = np.linalg.norm(wv)

    # with the above we can easily find C(X), now we need to go further an find
    # dC(X)/dX
    # on a per-triangle basis, this is a vector with 9 elements.
    # the first 3 elements correspond to the first vertex i, the next 3 to j and the last 3 to k
    # so dC(Xi)/dXi is a 3 component vector, however, because the (x,y,z) 
    # components are treatly identically, we only beed to find the expression for
    # one of them and then we can substitute x for y or z to get the others. 

    # find dCu(Xi.x)/d(Xi.x)

    # Equation (10) in Baraff-Witkin.
    Cu = area * (norm_wu - 1.0)

    dw_denominator = (delta_u1 * delta_v2 - delta_u2 * delta_v1)

    Xi_factor = (delta_v1 - delta_v2) / dw_denominator
    Xj_factor = delta_v2 / dw_denominator
    Xk_factor = -delta_v1 / dw_denominator

    wu_normalized = wu / norm_wu

    # It would have been nice if these derivates where give in the original paper,
    # sadly they weren't so I had to derive them myself by hand. I also found
    # this blog online with the derivation, however I was confused a bit about the 
    # notation at first, so I'll make my own blog post. 
    # blog: http://davidpritchard.org/freecloth/docs/report-single/

    dCu_Xi = area * Xi_factor * wu_normalized  # it would be slightly more effecient to multiply all the scalars first and only then to the vector multiply
    dCu_Xj = area * Xj_factor * wu_normalized
    dCu_Xk = area * Xk_factor * wu_normalized

    u_stiffness = v_stiffness = 1.0
    # Equation (7) in Baraff-Witkin.
    force_i = u_stiffness * Cu * dCu_Xi
    force_j = u_stiffness * Cu * dCu_Xi
    force_k = u_stiffness * Cu * dCu_Xi

    # print(force_i)
    # print(force_j)
    # print(force_k)

    # todo think about a lockfree way to fill force vector
    # force vector has 1 element per vertex, but most vertices are contained in multiple triangles
    # so if we compute the triangle forces in parallel, we need a lock when writing to the vertex position

    # maybe: one pass over triangles -> local storage per triangle

    # solution parallel pass over triangles -> store calculated forces locally
    # then sequential pass over triangles that copies triangle forces into forces vector

    # important abstraction later: function that fill in forces vector
    # almost all sims work with forces applied to particles
    # in BW98 via triangle based stretch condition (in Blender if mesh is passed with quads, just use quad face attribute on looptris)
    # in MSD sims via edge based hooke's law

    # also: function that fills force jacobians matrix